In [1]:
import numpy as np
import csv

In [2]:
Earthradius = 6371000
Sunradius = 695508000

This is the array where I'm going to store the planet objects.

In [3]:
kepid = []
koi = []
per = []
adivr = []
delta = []
derror = []

We want our planet sample to match that of Sanchis-Ojeda et al. (2014). They present 106 candidates, but their occurrence rate calculations focus on USP planets around G and K stars, that have effective temperatures between 4100 and 6100 K, log(g) between 4.0 and 4.9, and Kepler magnitudes below 16. The "constrain" function that I define applies these to a CSV, where I just have to enter the column numbers for each of these. The file that I open is Table 2 from  Sanchis-Ojeda et al. (2014), where we also constrain for orbital periods between 4 and 24 hours and radii between 0.84 and 4 Earth radii.

For each of these planets, we record the KIC, KOI, orbital period, transit probability, and transit depth along with its uncertainty.

In [4]:
def constrain(teff, logg, mkep):
    if ((row[teff] != "") and (4100 < float(row[teff]) < 6100)):
        if (((row[logg]) != "") and (4.0  < float(row[logg]) < 4.9)):
            if ((row[mkep] != "") and (float(row[mkep]) < 16)):
                return True

In [5]:
with open('USPCandidates.csv', encoding='latin-1') as File:
    reader = csv.reader(File, delimiter=',')
    rownum = 0
    for row in reader:
        if rownum > 0:
            if constrain(3,6,2):
                if ((row[12] != "") and (4 < (float(row[12])*24) < 24)):
                    if ((row[23] != "") and (.84 < (float(row[23])) < 4)):
                        kepid.append(int(row[0]))
                        koi.append(int(round(float(row[1]))))
                        per.append(float(row[12]))
                        adivr.append(float(row[20]))
                        delta.append(float(row[14]))
                        derror.append(float(row[15]))

        rownum += 1 
        
print(len(kepid))

72


In [6]:
falsepositives = [4929299, 10527135, 5642620, 12170648, 3834322,6750902,7269881,10186945,12115188]

todelete = []
for j in range(0,len(falsepositives)):
    for i in range(0, len(kepid)):
        if kepid[i] == falsepositives[j]:
            todelete.append(i)
            
todelete.reverse()
for k in todelete:
    del kepid[k]
    del koi[k]
    del per[k]
    del adivr[k]
    del delta[k]
    del derror[k]
print(len(kepid))

63


There are 72 planets that fit these criteria. Their paper says they use 69, so we're pretty close.

Now we have to calculate the occurrence rate of USP planets. To do this, we use the formula for Signal to Noise Ratio (SNR) from Howard et al. (2012), which is as follows:

\begin{equation}
    SNR = \frac{\delta}{\sigma_{CDPP}}\sqrt{\frac{T_{0}t}{(6\: hr)P_{orb}}}
\end{equation}

We calculate the SNR for each planet/host star pair.

We use the Kepler catalog to get the host star parameters, and record the Teff, log(g), and Fe/H.  

In [7]:
stars1 = []
snr = np.zeros(len(kepid))
teff= np.zeros(len(kepid))
logg = np.zeros(len(kepid))
feh = np.zeros(len(kepid))

with open('keplerstellar.csv') as File:
    reader = csv.reader(File, delimiter=',')
    rownum = 0
    for row in reader:
        if rownum > 0:
            if constrain(6,8,5):
                if "1" in (str(row[19])[:16]):
                    if ((row[1]) != "") and "q1_q16" in str(row[0]):
                        stars1.append(int(row[1])) 
                        if row[21] != "" and row[22] != "":
                            time = float(row[21])*float(row[22])
                            if row[23] != "":
                                for i in range(len(kepid)):
                                    if int(row[1]) == kepid[i]:
                                        t = (per[i]*24)*(1/adivr[i])*(1/np.pi) # transit duration
                                        snr[i] = ((delta[i]/(float(row[23])))*np.sqrt((t*time)/(per[i]*6)))
                                        teff[i] = (float(row[6]))
                                        logg[i] = (float(row[8]))
                                        feh[i] = (float(row[10]))
                                        
        rownum += 1
print("Total number of stars searched: ", len(stars1))

Total number of stars searched:  105266


The total number of stars searched matches pretty well the number reported in Sanchis-Ojeda et al. (2014).

The Kepler catalog really isn't the best place to get stellar parameters, so I'm replacing the values with ones from Winn et al. (2017) that use the updated CKS parameteters. But not all of the stars I need values for are in that population, so I'll just keep the Kepler values for those.

In [8]:
loggerr = np.zeros(len(kepid))
tefferr = np.zeros(len(kepid))
feherr = np.zeros(len(kepid))


with open('ajaa7b7ct1_ascii.csv') as File:
    reader = csv.reader(File, delimiter=',')
    rownum = 0
    for row in reader:
        if rownum > 2:
            for i in range(len(kepid)):
                if int(row[0][3:]) == koi[i]:
                    teff[i] = (float(row[1]))
                    tefferr[i] = np.max([(float(row[3])), (float(row[2]))])
                    logg[i] = (float(row[4]))
                    loggerr[i] = np.max([(float(row[6])), (float(row[5]))])
                    feh[i] = (float(row[7]))
                    feherr[i] = np.max([(float(row[9])), (float(row[8]))])
                elif int(row[0][3:]) == kepid[i]:
                    teff[i] = (float(row[1]))
                    tefferr[i] = np.max([(float(row[3])), (float(row[2]))])
                    logg[i] = (float(row[4]))
                    loggerr[i] = np.max([(float(row[6])), (float(row[5]))])
                    feh[i] = (float(row[7]))
                    feherr[i] = np.max([(float(row[9])), (float(row[8]))])
#                 if (int(row[0][3:]) == planets[i].koi):
#                     planets[i].Teff = (float(row[1]))
#                     planets[i].Tefferr = [(float(row[3])), (float(row[2]))]
#                     print(planets[i].Tefferr)
#                     planets[i].logg = (float(row[4]))
#                     planets[i].loggerr = [(float(row[6])), (float(row[5]))]
#                     planets[i].feh = (float(row[7]))
#                     planets[i].feherr = [(float(row[9])), (float(row[8]))]
        rownum += 1

In [9]:
print(len(tefferr))
print(len(kepid))
print(kepid)

63
63
[2718885, 3112129, 4665571, 8435766, 9642018, 9825174, 10585738, 11187332, 11550689, 1717722, 3444588, 4055304, 4144576, 5040077, 5095635, 5175986, 5513012, 5942808, 5972334, 6129524, 6265792, 6294819, 6310636, 6362874, 6867588, 6934291, 6964929, 6974658, 7102227, 7605093, 7749002, 8278371, 8558011, 8895758, 8947520, 9092504, 9149789, 9221517, 9456281, 9472074, 9473078, 9580167, 10024051, 10028535, 10319385, 10468885, 10647452, 10975146, 11030475, 11401182, 11547505, 11600889, 11752632, 11870545, 11904151, 12265786, 12405333, 5773121, 5980208, 8416523, 9353742, 9475552, 9967771]


One of the main new things we want to do is incorporate new Gaia parallaxes into the radius calculations. We used the gaia-kepler.fun crossmatch database created by Megan Bedell.

In [10]:
from astropy.table import Table
data = Table.read('kepler_dr2_1arcsec.fits', format='fits')

In [11]:
kepid1 = []
parallax2 = np.zeros(len(kepid))
perror2 = np.zeros(len(kepid))

for i in range(len(data['kepid'])):
    for j in range(len(kepid)):
        if kepid[j] == data['kepid'][i]:
            parallax2[j] = (data['parallax'][i])
            perror2[j] = (data['parallax_error'][i])
#         if (planets[j].kepid) == data['kepid'][i]:
#             planets[j].parallax = (data['parallax'][i])
#             planets[j].perror = (data['parallax_error'][i])

In [12]:
parallax3 = parallax2
print(len(parallax3))
print(np.where(parallax3 == 0))
#print(noparallax)
print(kepid[48])
print(len(kepid))
# del kepid[noparallax]
# del koi[noparallax]
# del per[noparallax]
# del teff[noparallax]
# del tefferr[noparallax]
# del logg[noparallax]
# del loggerr[noparallax]
# del feh[noparallax]
# del feherr[noparallax]
# del delta[noparallax]
# del derror[noparallax]
# del parallax2[noparallax]
# del perror2[noparallax]

# print(len(parallax2))

63
(array([48]),)
11030475
63


We need to remove this planet becasue it doesn't have a Gaia parallax.

In [13]:
koi = [koi[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
adivr = [adivr[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
per = [per[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
teff = [teff[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
logg = [logg[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
feh = [feh[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
tefferr = [tefferr[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
loggerr = [loggerr[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
feherr = [feherr[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
delta = [delta[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
derror = [derror[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
parallax2 = [parallax2[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
perror2 = [perror2[i] for i in range(0,len(kepid)) if kepid[i] != 11030475]
kepid = [i for i in kepid if i != 11030475] # there is no Gaia parallax for this KepID.

# del planets[51]
# print(len(planets))
#print(len(kepid4))
print(len(parallax3))

NameError: name 'kepid4' is not defined

In [14]:
print(len(parallax2))

62


Now, I'm going to record the photometric band magnitudes for each of the host stars. The bands are [Kepler, j, h, k, g, r, i, z].

In [17]:
kepid2 = np.zeros(len(kepid))
kepmag = np.zeros(len(kepid))
jmag = np.zeros(len(kepid))
hmag = np.zeros(len(kepid))
kmag = np.zeros(len(kepid))
gmag = np.zeros(len(kepid))
rmag = np.zeros(len(kepid))
imag = np.zeros(len(kepid))
zmag = np.zeros(len(kepid))

with open('kepler_search.csv') as File:
    reader = csv.reader(File, delimiter=',')
    rownum = 0
    for row in reader:
        if rownum > 1:
            if (int(row[0]) in kepid) and ((int(row[0]) in kepid2) == False):
                for i in range(len(kepid)):
                    if int(row[0]) == kepid[i]:
                        kepid2[i] = int(row[0])
                        kepmag[i] = float(row[37])
                        jmag[i] = float(row[34])
                        hmag[i] = float(row[35])
                        kmag[i] = float(row[36])
                        gmag[i] = float(row[30])
                        rmag[i] = float(row[31])
                        imag[i] = float(row[32])
                        zmag[i] = float(row[33])
#                     if int(row[0]) == planets[i].kepid:
#                         planets[i].photoarray = [float(row[37]), float(row[34]), float(row[35]), float(row[36]),
#                                                   float(row[30]), float(row[31]), float(row[32]), float(row[33])]
        rownum += 1
print(len(kepid2))

62


In [18]:
# for i in range(0,len(planets)):
#     if planets[i].photoarray == 'none':
#         print(planets[i].kepid)
        
# for i in range(0,len(planets)):
#     if planets[i].Tefferr == ['none', 'none']:
#         print(i)
print(tefferr[48])
print(teff[48])
print(logg[48])
print(loggerr[48])
print(feh[48])
print(feherr[48])
print(kepid[48])
print(perror2[48])

64.0
4776.0
4.6
0.03
-0.11
0.04
11401182
0.0174929977377


All of the stars have recorded photometric band magnitudes. Now we're going to use the isochrones package to interpolate the stellar host radii based on these band magnitudes, as well as log(g), Teff, and Fe/H.

In [19]:
from isochrones import StarModel
from isochrones.mist import MIST_Isochrone

bands = ['J','H','K','Kepler','g','r','i','z']
mist = MIST_Isochrone(bands = bands)
magradii = []
sigma = []
radarray = []
deltaarray = []
num_samples = 1000


In [21]:
file = open('PhotometryPlanetRadiiSamplesRedo2.csv', 'w')
with file:
    writer = csv.writer(file)
    for i in range(0, len(kepid2)):
        radiisamples = []
        mags = {'J':(jmag[i],0.05),'H':(hmag[i],0.05),'K':(kmag[i],0.05), 'Kepler':(kepmag[i],0.05), 'g':(gmag[i],0.05), 'r':(rmag[i],0.05), 'i':(imag[i],0.05), 'z':(zmag[i],0.05)}
#         mags = {'J':(planets[i].photoarray[1],0.05),'H':(planets[i].photoarray[2],0.05),'K':(planets[i].photoarray[3],0.05), 
#                 'Kepler':(planets[i].photoarray[0],0.05), 'g':(planets[i].photoarray[4],0.05), 'r':(planets[i].photoarray[5],0.05), 
#                 'i':(planets[i].photoarray[6],0.05), 'z':(planets[i].photoarray[7],0.05)}
#         if (planets[i].Tefferr != ['none', 'none']):
#             model = StarModel(mist, logg = (planets[i].logg,planets[i].loggerr[0]), teff = (planets[i].Teff, planets[i].Tefferr[0]),feh = (planets[i].feh,planets[i].feherr[0]), parallax = (planets[i].parallax,planets[i].perror), **mags)
#         else:
        if (tefferr[i] != 0):
            try:
                model = StarModel(mist, logg = (logg[i],loggerr[i]), teff = (teff[i], tefferr[i]),feh = (feh[i],feherr[i]), parallax = (parallax2[i],perror2[i]), **mags)
            except:
                model = StarModel(mist, parallax = (parallax2[i], perror2[i]), **mags)    
        else:
            model = StarModel(mist, parallax = (parallax2[i], perror2[i]), **mags)
        StarModel.fit_multinest(model,n_live_points=1000, basename=None, verbose=False, refit=True, overwrite=True,test=False)
        magradii.append(model.samples.radius_0_0.quantile(0.5))
        ##planets[i].starrad = model.samples.radius_0_0.quantile(0.5)
        sigma.append(np.std(model.samples.radius_0_0))
        ##planets[i].starraderr = np.std(model.samples.radius_0_0)
        radarray = np.random.choice(model.samples.radius_0_0, num_samples) ## star radii
        deltaarray = np.random.normal(delta[i],derror[i], num_samples)
        for j in range(len(radarray)):
            if deltaarray[j] > 0:
                radiisamples.append((radarray[j]*Sunradius*np.sqrt(deltaarray[j]/1000000))/Earthradius)
            else:
                radiisamples.append("nan")
        row = [kepid[i],radiisamples]
        writer.writerow(row)
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [ ]:
print(kepid)